## Preprocessing

In [26]:
import pandas as pd
import re
import spacy

import torch
from torch_geometric.data import HeteroData

import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

In [9]:
filename = 'data/arxiv-metadata-oai-snapshot-10000.csv'
df = pd.read_csv(filename)

In [10]:
def extract_pages(s):
    match = re.search(r"(\d+)\s*pages", s)
    if match:
        return int(match.group(1))
    else:
        return None

In [11]:
df['authors_parsed'] = df['authors_parsed'].apply(lambda x: [" ".join(i).strip() for i in eval(x)])
df['versions'] = df['versions'].apply(lambda x: eval(x)[0]["created"])
df['timestamp'] = pd.to_datetime(df['versions'], format="%a, %d %b %Y %H:%M:%S %Z")
df['timestamp'] = df['timestamp'].apply(lambda x: x.timestamp())
df["categories"] = df["categories"].apply(lambda x: x.split(" "))
df.drop(columns=["submitter", "versions", "update_date", "authors"], inplace=True)
df["pages"] = df.comments.apply(lambda x: extract_pages(str(x)))
df.head()

,id,title,comments,journal-ref,doi,report-no,categories,license,abstract,authors_parsed,timestamp,pages
0,704.0001,Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,[hep-ph],NaN,A fully differential calculation in perturba...,"[Balázs C., Berger E. L., Nadolsky P. M., Yuan...",1.175542e+09,37.0
1,704.0002,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,NaN,NaN,NaN,"[math.CO, cs.CG]",http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[Streinu Ileana, Theran Louis]",1.175308e+09,NaN
2,704.0003,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",NaN,NaN,NaN,[physics.gen-ph],NaN,The evolution of Earth-Moon system is descri...,[Pan Hongjun],1.175460e+09,23.0
3,704.0004,A determinant of Stirling cycle numbers counts...,11 pages,NaN,NaN,NaN,[math.CO],NaN,We show that a determinant of Stirling cycle...,[Callan David],1.175311e+09,11.0
4,704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,NaN,"Illinois J. Math. 52 (2008) no.2, 681-689",NaN,NaN,"[math.CA, math.FA]",NaN,In this paper we show how to compute the $\L...,"[Abu-Shammala Wael, Torchinsky Alberto]",1.175537e+09,NaN


In [12]:
def tokenize_and_normalize(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [token.text.lower() for token in doc if not token.is_punct]

def lemm(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_punct]

In [13]:
df_short = df[:10]

In [14]:
tokenized_words_list = []
lemm_list = []


for index, row in df_short.iterrows():
    text = row['abstract']
    tokenized_words = tokenize_and_normalize(text)
    lemm_words = lemm(text)
    tokenized_words_list.append(tokenized_words)
    lemm_list.append(lemm_words)


print(tokenized_words_list)
print(lemm_list)


[['  ', 'a', 'fully', 'differential', 'calculation', 'in', 'perturbative', 'quantum', 'chromodynamics', 'is', '\r\n', 'presented', 'for', 'the', 'production', 'of', 'massive', 'photon', 'pairs', 'at', 'hadron', 'colliders', 'all', '\r\n', 'next', 'to', 'leading', 'order', 'perturbative', 'contributions', 'from', 'quark', 'antiquark', '\r\n', 'gluon-(anti)quark', 'and', 'gluon', 'gluon', 'subprocesses', 'are', 'included', 'as', 'well', 'as', '\r\n', 'all', 'orders', 'resummation', 'of', 'initial', 'state', 'gluon', 'radiation', 'valid', 'at', '\r\n', 'next', 'to', 'next', 'to', 'leading', 'logarithmic', 'accuracy', 'the', 'region', 'of', 'phase', 'space', 'is', '\r\n', 'specified', 'in', 'which', 'the', 'calculation', 'is', 'most', 'reliable', 'good', 'agreement', 'is', '\r\n', 'demonstrated', 'with', 'data', 'from', 'the', 'fermilab', 'tevatron', 'and', 'predictions', 'are', 'made', 'for', '\r\n', 'more', 'detailed', 'tests', 'with', 'cdf', 'and', 'do', 'data', 'predictions', 'are', 's

In [15]:
print(tokenized_words_list[0])

['  ', 'a', 'fully', 'differential', 'calculation', 'in', 'perturbative', 'quantum', 'chromodynamics', 'is', '\r\n', 'presented', 'for', 'the', 'production', 'of', 'massive', 'photon', 'pairs', 'at', 'hadron', 'colliders', 'all', '\r\n', 'next', 'to', 'leading', 'order', 'perturbative', 'contributions', 'from', 'quark', 'antiquark', '\r\n', 'gluon-(anti)quark', 'and', 'gluon', 'gluon', 'subprocesses', 'are', 'included', 'as', 'well', 'as', '\r\n', 'all', 'orders', 'resummation', 'of', 'initial', 'state', 'gluon', 'radiation', 'valid', 'at', '\r\n', 'next', 'to', 'next', 'to', 'leading', 'logarithmic', 'accuracy', 'the', 'region', 'of', 'phase', 'space', 'is', '\r\n', 'specified', 'in', 'which', 'the', 'calculation', 'is', 'most', 'reliable', 'good', 'agreement', 'is', '\r\n', 'demonstrated', 'with', 'data', 'from', 'the', 'fermilab', 'tevatron', 'and', 'predictions', 'are', 'made', 'for', '\r\n', 'more', 'detailed', 'tests', 'with', 'cdf', 'and', 'do', 'data', 'predictions', 'are', 'sh

In [16]:
for i in range(10):
    a = len(tokenized_words_list[i])
    print(a)

165
132
164
42
45
161
172
149
323
118


In [25]:
data = HeteroData()

# Initialisiere die Anzahl der Knoten für den 'paper'-Typ
data['paper'].num_nodes = 0

# Initialisiere die Feature-Liste im HeteroData-Objekt
data['x'] = []

# Füge Knoten für Paper hinzu
for idx, tokenized_words in enumerate(tokenized_words_list):
    # Erzeuge einen eindeutigen Namen für jeden Paper-Knoten
    paper_node_name = f'paper_{idx}'
    
    # Füge den Paper-Knoten zum HeteroData-Graphen hinzu
    data['paper'].num_nodes += 1
    data['x'].append(tokenized_words)

# Ausgabe des erstellten HeteroData-Graphen
print(data)

HeteroData(
  x=[10],
  paper={ num_nodes=10 }
)


In [30]:
edges = [(src, dst) for src, dst in zip(data['edge_index'][0], data['edge_index'][1])]
node_types = [node_type.split('_')[0] for node_type in data.keys()]

# Erzeuge einen gerichteten MultiDiGraph
graph = nx.MultiDiGraph()

# Füge Knoten und Kanten zum Graphen hinzu
graph.add_nodes_from(node_types)
graph.add_edges_from(edges)

# Zeichne den Graphen
pos = nx.spring_layout(graph)  # oder eine andere Layout-Option
nx.draw(graph, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=800, arrowsize=20)
plt.show()

KeyError: 0

In [1]:

data = HeteroData()

data['paper'].num_nodes = 300
data['author'].num_nodes = 200
data['paper', 'written_by', 'author'].edge_index = torch.tensor([
    [1,2,2],
    [100,102,105]
]) 

1,100
2,102
data['author'].x = torch.tensor([
    [3,4,1],
    [1,1,1],
    [0,0,0]
])


data['paper', 'written_by', 'author'].edge_attr = torch.tensor([
    [3,4,1],
    [1,1,1],
    [0,0,0]
])

/home/amos/miniconda3/envs/GNNpapersearch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data

HeteroData(
  paper={ num_nodes=300 },
  author={
    num_nodes=200,
    x=[3, 3],
  },
  (paper, written_by, author)={
    edge_index=[2, 3],
    edge_attr=[3, 3],
  }
)

In [3]:
import torch_geometric.transforms as T

transform = T.Compose([
       T.RemoveIsolatedNodes(),
       T.ToUndirected(merge=False), # don't merge reversed edges into the original edge type
       T.RemoveDuplicatedEdges(),
])
data_before = data
data = transform(data)

In [4]:
data

HeteroData(
  paper={ num_nodes=2 },
  author={
    num_nodes=3,
    x=[3, 3],
  },
  (paper, written_by, author)={
    edge_index=[2, 3],
    edge_attr=[3, 3],
  },
  (author, rev_written_by, paper)={
    edge_index=[2, 3],
    edge_attr=[3, 3],
  }
)

In [50]:
data['paper', 'written_by', 'author'].edge_index

tensor([[0, 1, 1],
        [0, 1, 2]])

In [12]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

NameError: name 'spacy' is not defined